In [1]:
import os

In [2]:
os.chdir("/Users/vladimirkalajcidi/endtoend")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    size: str

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            size=self.params.SIZE,
        )

        return prepare_base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import whisper
import torch
from transformers import WhisperForConditionalGeneration

In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = WhisperForConditionalGeneration.from_pretrained(self.config.size)

        self.save_model(path=self.config.base_model_path, model=self.model)

        self.save_model(path=self.config.updated_base_model_path, model=self.model)
    

    @staticmethod
    def save_model(path: Path, model):
        model.save_pretrained(path, from_pt=True)

In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-08-10 15:48:50,508: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-10 15:48:50,510: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-10 15:48:50,511: INFO: common: created directory at: artifacts]
[2024-08-10 15:48:50,512: INFO: common: created directory at: artifacts/prepare_base_model]
